# 75.06/95.58 Organización de Datos
# Primer Cuatrimestre de 2020
# Trabajo Práctico 1: Enunciado

El trabajo práctico 1 de la materia se basa en el análisis de los tweets del set de datos de la competencia: https://www.kaggle.com/c/nlp-getting-started.  

- id - identificador unico para cada  tweet
- text - el texto del tweet
- location - ubicación desde donde fue enviado (podría no estar)
- keyword - un keyword para el tweet  (podría faltar)
- target - en train.csv, indica si se trata de un desastre real  (1) o no (0)

El objetivo del primer TP es realizar un análisis exploratorio del set de datos. Queremos ver qué cosas podemos descubrir sobre los datos que puedan resultar interesantes. Estas cosas pueden estar relacionadas al objetivo del TP2 (predecir si un cierto tweet es real o no) o no, ambas son de interés.

## Los requisitos de la primera entrega son los siguientes:

- El análisis debe estar hecho en Python Pandas o R.
- El análisis debe entregarse en formato pdf vía gradescope. En el informe no va código.
- Informar el link a un repositorio Github en donde pueda bajarse el código completo para generar el análisis.

## La evaluación del TP se realizará en base al siguiente criterio:

- Originalidad del análisis exploratorio. 
- Calidad del reporte. ¿Está bien escrito? ¿Es claro y preciso? 
- Calidad del análisis exploratorio: qué tipo de preguntas se hacen y de qué forma se responden, ¿es la respuesta clara y concisa con respecto a la pregunta formulada? 
- Calidad de las visualizaciones presentadas.
- ¿Tienen todos los ejes su rótulo?
- ¿Tiene cada visualización un título?
- ¿Es entendible la visualización sin tener que leer la explicación?
- ¿El tipo de plot elegido es adecuado para lo que se quiere visualizar?
- ¿Es una visualización interesante?
- ¿El uso del color es adecuado?
- ¿Hay un exceso o falta de elementos visuales en la visualización elegida?
- ¿La visualización es consistente con los datos?
- Conclusiones presentadas.
- ¿Presenta el grupo un listado de "insights" aprendidos sobre los datos en base al análisis realizado? ¿Es interesante? 


El grupo que realice el mejor análisis exploratorio obtendrá 10 puntos para cada uno de sus integrantes que podrán ser usados en el parcial además de ser publicado en el repositorio de la materia como ejemplo para los siguientes cuatrimestres. 



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

pd.options.display.float_format = '{:20,.2f}'.format 

import warnings
warnings.filterwarnings('ignore')

## Empiezo a analizar la informacion

### Analisis de realacion tweets verdaderos/falsos por longitud de texto

In [31]:
tweets_original = pd.read_csv('train.csv',index_col=['id'])
tweets_original.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [32]:
tweets = tweets_original.drop(columns=['keyword', 'location'])
tweets['len_tw'] = tweets['text'].str.len()
tweets.head()

,text,target,len_tw
id,,,
1,Our Deeds are the Reason of this #earthquake M...,1,69
4,Forest fire near La Ronge Sask. Canada,1,38
5,All residents asked to 'shelter in place' are ...,1,133
6,"13,000 people receive #wildfires evacuation or...",1,65
7,Just got sent this photo from Ruby #Alaska as ...,1,88


Para una mejor visualizacion del target, en vez de tener numeros asociados a la veracidad de los mismos, pondremos strings de tal forma que sean mas descriptivos. Entonces al valor 1 le cambiamos por "Real" y al valor 0 lo cambiamos por "No Real". De esta forma esta columna ahora es mucho mas descriptiva que un valor numerico.

In [33]:
tweets['target'] = tweets['target'].replace({0: 'No Real', 1: 'Real'})
tweets.head()


,text,target,len_tw
id,,,
1,Our Deeds are the Reason of this #earthquake M...,Real,69
4,Forest fire near La Ronge Sask. Canada,Real,38
5,All residents asked to 'shelter in place' are ...,Real,133
6,"13,000 people receive #wildfires evacuation or...",Real,65
7,Just got sent this photo from Ruby #Alaska as ...,Real,88


Vemos la cantidad de tweets para cada caso de target.

In [34]:
tweets['target'].value_counts()

No Real    4342
Real       3271
Name: target, dtype: int64

Ahora agrupamos a los tweets por su target para hacerle algunos calculos estadisticos

In [35]:
grouped_by_target = tweets.groupby(['target']).agg({'len_tw': ['mean', 'count']})
grouped_by_target.head()

len_tw      
                        mean count
target                            
No Real                95.71  4342
Real                  108.11  3271

Juntamos ambos levels de indice, para que queden 2 columnas

In [36]:
level0 = grouped_by_target.columns.get_level_values(0)
level1 = grouped_by_target.columns.get_level_values(1)
grouped_by_target.columns = level0 + '_' + level1
grouped_by_target.head()

,len_tw_mean,len_tw_count
target,,
No Real,95.71,4342
Real,108.11,3271


In [37]:
grouped_by_target.reset_index(inplace=True)
grouped_by_target.head()

,target,len_tw_mean,len_tw_count
0,No Real,95.71,4342
1,Real,108.11,3271


### Analisis de realacion tweets verdaderos/falsos por localizacion


Le quito los nulos

In [72]:
tweet_sin_nulos = tweets_original.dropna()
tweet_sin_nulos.head()

,keyword,location,text,target
id,,,,
48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [73]:
grupo_por_localizacion = tweet_sin_nulos.groupby(['location']).agg({'target':['count','sum']})
grupo_por_localizacion

target    
                        count sum
location                         
                            1   1
  Glasgow                   1   0
  Melbourne, Australia      1   0
  News                      1   1
  å_                        1   0
...                       ...  ..
å_: ?? ÌÑ ? : ?             1   0
å_å_Los Mina Cityã¢        1   0
å¡å¡Midwest Û¢Û¢          1   0
åÊ(?Û¢`?Û¢å«)??           1   0
åø\_(?)_/åø                 1   1

[3341 rows x 2 columns]

Juntamos ambos levels de indice, para que queden 2 columnas

In [74]:
level0 = grupo_por_localizacion.columns.get_level_values(0)
level1 = grupo_por_localizacion.columns.get_level_values(1)
grupo_por_localizacion.columns = level0 + '_' + level1
grupo_por_localizacion.head()

,target_count,target_sum
location,,
,1,1
Glasgow,1,0
"Melbourne, Australia",1,0
News,1,1
å_,1,0


Dada la cantidad de locaciones distintas los divido en aquellas locaciones con mas de la mitad de tweets verdaderos y aquellas que no.

In [86]:
grupo_por_localizacion['cantidad_de_localizaciones'] = 1
grupo_por_localizacion.loc[grupo_por_localizacion['target_sum']/grupo_por_localizacion['target_count'] < 0.5 , 'mayoria'] = 'Falsos'
grupo_por_localizacion.loc[grupo_por_localizacion['target_sum']/grupo_por_localizacion['target_count'] > 0.5 , 'mayoria'] = 'Verdaderos'
grupo_por_localizacion.loc[grupo_por_localizacion['target_sum']/grupo_por_localizacion['target_count'] == 0.5 , 'mayoria'] = 'Neutros'
agrupacion_por_mayoria_en_localizacion = grupo_por_localizacion.groupby(['mayoria']).agg({'target_count':['sum'],'target_sum':['sum'],'cantidad_de_localizaciones':['sum']})
agrupacion_por_mayoria_en_localizacion.columns = ['cantidad_de_tweets','tweets_verdaderos','cantidad_de_localizaciones']
agrupacion_por_mayoria_en_localizacion

,cantidad_de_tweets,tweets_verdaderos,cantidad_de_localizaciones
mayoria,,,
Falsos,2665,204,1919
Neutros,386,193,136
Verdaderos,2029,1799,1286


### Analisis de realacion tweets localizacion y longitud de textos